In [3]:
from sklearn.model_selection import cross_validate

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
train = pd.read_csv('train.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
test = train.copy()

In [8]:
import math

In [9]:
train['Initial']=0
for i in train:
  train['Initial'] = train.Name.str.extract('([A-Za-z]+)\.')

test['Initial']=0
for i in test:
  test['Initial'] = test.Name.str.extract('([A-Za-z]+)\.')

In [10]:
train.groupby('Initial')['Age'].mean()

Initial
Capt        70.000000
Col         58.000000
Countess    33.000000
Don         40.000000
Dr          42.000000
Jonkheer    38.000000
Lady        48.000000
Major       48.500000
Master       4.574167
Miss        21.773973
Mlle        24.000000
Mme         24.000000
Mr          32.368090
Mrs         35.898148
Ms          28.000000
Rev         43.166667
Sir         49.000000
Name: Age, dtype: float64

In [11]:
test.groupby('Initial')['Age'].mean()

Initial
Capt        70.000000
Col         58.000000
Countess    33.000000
Don         40.000000
Dr          42.000000
Jonkheer    38.000000
Lady        48.000000
Major       48.500000
Master       4.574167
Miss        21.773973
Mlle        24.000000
Mme         24.000000
Mr          32.368090
Mrs         35.898148
Ms          28.000000
Rev         43.166667
Sir         49.000000
Name: Age, dtype: float64

In [12]:
train.loc[(train.Age.isnull())&(train.Initial=='Mr'),'Age']=33
train.loc[(train.Age.isnull())&(train.Initial=='Mrs'),'Age']=36
train.loc[(train.Age.isnull())&(train.Initial=='Master'),'Age']=5
train.loc[(train.Age.isnull())&(train.Initial=='Miss'),'Age']=22
train.loc[(train.Age.isnull())&(train.Initial=='Dr'),'Age']=42

test.loc[(test.Age.isnull())&(test.Initial=='Mr'),'Age']=33
test.loc[(test.Age.isnull())&(test.Initial=='Mrs'),'Age']=36
test.loc[(test.Age.isnull())&(test.Initial=='Master'),'Age']=5
test.loc[(test.Age.isnull())&(test.Initial=='Miss'),'Age']=22
test.loc[(test.Age.isnull())&(test.Initial=='Dr'),'Age']=42

In [13]:
train.Age.isnull().any(),test.Age.isnull().any()

(False, False)

In [14]:
data = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]
target = train['Survived']

In [15]:
data=pd.get_dummies(data=data, columns=['Sex'], prefix=['Sex'])

In [16]:
data=pd.get_dummies(data=data, columns=['Embarked'], prefix=['Embarked'])

In [17]:
data['Age'] = data['Age'].fillna(math.floor(data['Age'].mean()))

In [18]:
data

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,0,1,0,0,1
1,1,38.0,1,0,1,0,1,0,0
2,3,26.0,0,0,1,0,0,0,1
3,1,35.0,1,0,1,0,0,0,1
4,3,35.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,0,1,0,0,1
887,1,19.0,0,0,1,0,0,0,1
888,3,22.0,1,2,1,0,0,0,1
889,1,26.0,0,0,0,1,1,0,0


In [19]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=800)

In [20]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9062454907039126 0.8187136806855116


In [21]:
import lightgbm as lgb

In [22]:
clf = lgb.LGBMClassifier()
clf.fit(train_input, train_target)

LGBMClassifier()

In [23]:
y_pred = clf.predict(test_input)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
accuracy = accuracy_score(y_pred, test_target)

In [26]:
print(accuracy_score(test_target, y_pred))

0.8156424581005587


In [27]:
print(clf.score(train_input, train_target))

0.9058988764044944


In [28]:
print(clf.score(test_input, test_target))

0.8156424581005587


In [29]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9487336971602996 0.811681276470009


In [30]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.13417364 0.36936216 0.06989715 0.04953311 0.16677811 0.16822465
 0.01392802 0.00824188 0.01986129]


In [31]:
data.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [32]:
data2 = data.drop(['SibSp'],axis =1)
data2

,Pclass,Age,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,0,0,1,0,0,1
1,1,38.0,0,1,0,1,0,0
2,3,26.0,0,1,0,0,0,1
3,1,35.0,0,1,0,0,0,1
4,3,35.0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,1,0,0,1
887,1,19.0,0,1,0,0,0,1
888,3,22.0,2,1,0,0,0,1
889,1,26.0,0,0,1,1,0,0


In [33]:
train_input, test_input, train_target, test_target = train_test_split(
    data2, target, test_size=0.2, random_state=800)

In [34]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8914963154811458 0.8229685807150595


In [35]:
lgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.13417364 0.36936216 0.06989715 0.04953311 0.16677811 0.16822465
 0.01392802 0.00824188 0.01986129]


In [36]:
data3 = data.drop(['Parch'],axis =1)
data3

,Pclass,Age,SibSp,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,0,0,1
1,1,38.0,1,1,0,1,0,0
2,3,26.0,0,1,0,0,0,1
3,1,35.0,1,1,0,0,0,1
4,3,35.0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,1,0,0,1
887,1,19.0,0,1,0,0,0,1
888,3,22.0,1,1,0,0,0,1
889,1,26.0,0,0,1,1,0,0


In [37]:
train_input, test_input, train_target, test_target = train_test_split(
    data3, target, test_size=0.2, random_state=800)

In [45]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8999241513273517 0.8243573328080369


In [51]:
lgb.fit(train_input, train_target)
print(lgb.feature_importances_)

[ 245 1596  225   88   41   43   45  132]


In [52]:
data3.columns

Index(['Pclass', 'Age', 'SibSp', 'Sex_female', 'Sex_male', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [53]:
sorted(lgb.feature_importances_)

[41, 43, 45, 88, 132, 225, 245, 1596]

In [39]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42,scale_pos_weight = 1.1)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8981685320506893 0.8299812863193144


In [40]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42,scale_pos_weight = 1.1, n_estimators = 70)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8886880646255356 0.8341869398207425


In [41]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42,max_depth=25, subsample = 0.8229, scale_pos_weight = 1.1, n_estimators = 70)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8886880646255356 0.8341869398207425


In [42]:
lgbm_parameter_bounds = {
                      'n_estimators' : (30,100),
                      'max_depth' : (1,3), # 나무의 깊이
                      'subsample' : (0.5,1)
                      }


In [43]:
import numpy as np
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(lgbm_parameter_bounds)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

nan nan


C:\Users\tj\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\tj\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tj\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "C:\Users\tj\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "C:\Users\tj\an